In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import sys
from pathlib import Path
# Add the project root to the Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

In [4]:
from langgraph.graph import add_messages
from langchain.tools import tool
from langchain.messages import ToolMessage, HumanMessage, AnyMessage
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition
from scripts.retrieve import load_retriever
from utils.utils import format_context
from reranker.rrf import ReciprocalRankFusion
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_core.documents import Document
from config import output_path_prefix
import pickle

with open(f"{output_path_prefix}_split_documents.pkl", "rb") as f:
        split_documents = pickle.load(f)

@tool
def retriever(query: str) -> list[Document]:
    """Retrieve documents from the vector database.

    Args:
        query: The query to retrieve documents from the vector database.
    """
    embeddings = UpstageEmbeddings(model="embedding-passage")
    bm25_retriever, faiss_retriever = load_retriever(split_documents, embeddings, kiwi=False, search_k=10)
    retrieved_docs_faiss = faiss_retriever.invoke(query)
    retrieved_docs_bm25 = bm25_retriever.invoke(query)
    retrieved_docs_faiss = ReciprocalRankFusion.calculate_rank_score(retrieved_docs_faiss)
    retrieved_docs_bm25 = ReciprocalRankFusion.calculate_rank_score(retrieved_docs_bm25)
    retrieved_docs = retrieved_docs_faiss + retrieved_docs_bm25
    rrf_docs = ReciprocalRankFusion.get_rrf_docs(retrieved_docs, cutoff=4)
    context = format_context(rrf_docs)

    return {"documents": rrf_docs, "context": context}


tools = [retriever]
tools_by_name = {tool.name: tool for tool in tools}

def tool_node(state: dict):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result, "documents": observation["documents"], "context": observation["context"]}

In [7]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Literal

In [6]:
llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

In [8]:
class Route(BaseModel):
    step: Literal["vector", "web_search"] = Field(
        None, description="Given a user question choose to route it to web search or a vectorstore."
    )

router = llm.with_structured_output(Route)

In [ ]:
ROUTE_PROMPT = """
You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use web-search.
"""